## Implementing Logistic Regression Manually

#### Load the necessary libraries

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

#### Load the dataset

In [31]:
data = pd.read_csv('email.csv')
data.head()

,x0,x1,x2,x3,x4,x5,class
0,1,1,1,0,1,1,1
1,1,0,0,1,1,0,0
2,1,0,1,1,0,0,1
3,1,1,0,0,1,0,0
4,1,1,0,1,0,1,1


#### Separate features and class

In [32]:
# shuffle the dataset
data = data.sample(frac = 1)

x = data.iloc[:, 1:6]
y = data.iloc[:, -1:]

#### Separate training and testing dataset

In [33]:
train_size = int(0.7 * len(x))
test_size = len(x) - train_size

x_train, y_train = x.iloc[:train_size], y.iloc[:train_size]
x_test, y_test = x.iloc[train_size:], y.iloc[train_size:]

# training data set
x_train = x_train.values
y_train = y_train.values.flatten()

# testing data set
x_test = x_test.values
y_test = y_test.values.flatten()

x_train

array([[0, 0, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 1, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0]], dtype=int64)

#### Define the sigmoid function

In [34]:
def sigmoid(z):
    g = 1 / (1 + np.exp(-z))
    return g

#### Define the cost function

$$loss(f(\mathbf{x}^{(i)}), y^{(i)}) = -y^{(i)} \log\left(f\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f\left( \mathbf{x}^{(i)} \right) \right) $$

In [35]:
def compute_cost(x, y, w, b):
    cost = 0
    m = len(y)
    
    for i in range(m):
        z_i = np.dot(x[i], w) + b
        y_hat = sigmoid(z_i)
        
        cost += -y[i] * np.log10(y_hat) + (1 - y[i]) * np.log10(1 - y_hat)
    
    cost = cost / m
    
    return cost

#### Compute Gradient

$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})
\end{align*}$$

In [36]:
def compute_gradient(x, y, w, b):
    m, n = x.shape
    dj_dw = np.zeros((n, ))
    dj_db = 0
    
    for i in range(m):
        f_i = sigmoid(np.dot(x[i], w) + b)
        err_i = f_i - y[i]
        
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * x[i, j]
        
        dj_db = dj_db + err_i
        
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    
    return dj_db, dj_dw

#### Implement Gradient Descent

$$\begin{align*}
&\text{repeat until convergence:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

In [37]:
def gradient_descent(x, y, w_in, b_in, alpha, epochs):
    
    cost_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    
    for i in range(epochs):
        
        # calculate gradient and update parameters
        dj_db, dj_dw = compute_gradient(x, y, w, b)
        
        # update parameters
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        cost_history.append(compute_cost(x, y, w, b))
        
        # print cost after iterations
        if (i % math.ceil(epochs / 10)) == 0:
            print("Iteration", i, ": Cost", cost_history[-1])
            
    return w, b, cost_history

#### Run gradient descent on training set

In [38]:
# initialize the weights
w_tmp = np.zeros_like(x_train[0])
b_tmp = 0

alpha = 0.1
epochs = 1000

w_out, b_out, _ = gradient_descent(x_train, y_train, w_tmp, b_tmp, alpha, epochs)

print("\nUpdated Parameters :\nw =", w_out, "\nb =", b_out)

Iteration 0 : Cost -0.04493619060897102
Iteration 100 : Cost 0.004990705677793
Iteration 200 : Cost 0.003046855490899157
Iteration 300 : Cost 0.0023646915786415513
Iteration 400 : Cost 0.002008043742488879
Iteration 500 : Cost 0.0017813494469775765
Iteration 600 : Cost 0.0016195263851390854
Iteration 700 : Cost 0.0014949971098055877
Iteration 800 : Cost 0.0013942191171873984
Iteration 900 : Cost 0.0013097877156859163

Updated Parameters :
w = [-0.37718249  2.38965929 -0.56101955 -3.52826194  4.18477583] 
b = 0.20577045645435854


#### Predict the probability

In [39]:
def predict(x, w, b):
    predictions = []
    m = x.shape[0]
    
    for i in range(m):
        z = np.dot(x[i], w) + b
        predictions.append(sigmoid(z))
        
    return np.array(predictions)

#### Predict values for test dataset

In [40]:
y_predict = predict(x_test, w_out, b_out)

# round off to nearest integer for binary classification
y_predict = np.rint(y_predict).astype(int)

#### Find accuracy of predictions

In [41]:
def calculate_accuracy(y, y_hat):
    m = len(y)
    count = 0
    
    for i in range(m):
        if (y[i] == y_hat[i]):
            count += 1
            
    accuracy = count / m * 100
    print("Accuracy =", accuracy, "%")

In [42]:
calculate_accuracy(y_test, y_predict)

Accuracy = 100.0 %


## Implementing Logistic Regression using sklearn

#### Load necessary libraries

In [44]:
from sklearn.linear_model import LogisticRegression

#### Build the model

In [46]:
classifier = LogisticRegression()

classifier.fit(x_train, y_train)

LogisticRegression()

#### Predict the values of test dataset

In [47]:
y_predictions = classifier.predict(x_test)

#### Find the accuracy

In [50]:
calculate_accuracy(y_test, y_predictions)

Accuracy = 100.0 %
